# Fear Behavior with Human Detection

**Step 1: Create display widgets**

In [ ]:
import ipywidgets.widgets as widgets
from IPython.display import display
import cv2

# Create widgets for displaying images
display_color = widgets.Image(format='jpeg', width='30%')
display_depth = widgets.Image(format='jpeg', width='30%')
layout = widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth], layout=layout)
display(sidebyside)

# Convert numpy array to jpeg coded data for displaying
def bgr8_to_jpeg(value):
    return bytes(cv2.imencode('.jpg', value)[1])

**Step 2: Load YOLO model for human detection**

In [ ]:
from ultralytics import YOLO

# Load the YOLO model
model = YOLO("yolo11l_half.engine")

**Step 3: Initialize camera and implement fear behavior with human detection**

The robot will:
- Detect humans using YOLO
- Calculate distance to detected humans using depth image
- Retreat at different speeds based on distance
- Rotate away if human is too close

In [ ]:
import traitlets
import numpy as np
import pyzed.sl as sl
import threading
import motors
from traitlets.config.configurable import SingletonConfigurable

# Initialize the Robot class
robot = motors.MotorsYukon(mecanum=False)

class Camera(SingletonConfigurable):
    color_value = traitlets.Any()  # Monitor color_value variable
    
    def __init__(self):
        super(Camera, self).__init__()

        self.zed = sl.Camera()
        init_params = sl.InitParameters()
        init_params.camera_resolution = sl.RESOLUTION.VGA
        init_params.depth_mode = sl.DEPTH_MODE.ULTRA
        init_params.coordinate_units = sl.UNIT.MILLIMETER

        status = self.zed.open(init_params)
        if status != sl.ERROR_CODE.SUCCESS:
            print("Camera Open : "+repr(status)+". Exit program.")
            self.zed.close()
            exit(1)

        self.runtime = sl.RuntimeParameters()
        self.thread_runnning_flag = False

        camera_info = self.zed.get_camera_information()
        self.width = camera_info.camera_configuration.resolution.width
        self.height = camera_info.camera_configuration.resolution.height
        self.image = sl.Mat(self.width, self.height, sl.MAT_TYPE.U8_C4, sl.MEM.CPU)
        self.depth = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C1, sl.MEM.CPU)
        self.point_cloud = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C4, sl.MEM.CPU)

    def _capture_frames(self):
        while(self.thread_runnning_flag == True):
            if self.zed.grab(self.runtime) == sl.ERROR_CODE.SUCCESS:
                
                # Retrieve Left image
                self.zed.retrieve_image(self.image, sl.VIEW.LEFT)
                # Retrieve depth map
                self.zed.retrieve_measure(self.depth, sl.MEASURE.DEPTH)
    
                self.color_value_BGRA = self.image.get_data()
                self.color_value = cv2.cvtColor(self.color_value_BGRA, cv2.COLOR_BGRA2BGR)
                self.depth_image = np.asanyarray(self.depth.get_data())
                
    def start(self):
        if self.thread_runnning_flag == False:
            self.thread_runnning_flag = True
            self.thread = threading.Thread(target=self._capture_frames)
            self.thread.start()

    def stop(self):
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False
            self.thread.join()
            robot.stop()

camera = Camera()
camera.start()

**Step 4: Implement fear behavior with human detection callback**

In [ ]:
def fear_callback(change):
    frame = change['new']
    
    # Run YOLO inference to detect humans
    results = model(frame, verbose=False)
    
    # Prepare depth colormap for display
    depth_colormap = cv2.applyColorMap(
        cv2.convertScaleAbs(camera.depth_image, alpha=0.03), 
        cv2.COLORMAP_JET)
    
    conf_threshold = 0.5
    human_detected = False
    min_distance = float('inf')
    closest_bbox = None
    
    # Check for human detections (class 0)
    for result in results:
        for i in range(len(result.boxes.cls)):
            if result.boxes.cls[i] == 0:  # Human subject
                if result.boxes.conf[i] > conf_threshold:
                    human_detected = True
                    bbox = result.boxes.xyxy[i]
                    
                    # Draw detection box
                    cv2.rectangle(frame, 
                                (int(bbox[0]), int(bbox[1])), 
                                (int(bbox[2]), int(bbox[3])), 
                                (255, 0, 0), 2)
                    
                    # Calculate center of bounding box
                    center_x = int((bbox[0] + bbox[2]) / 2)
                    center_y = int((bbox[1] + bbox[3]) / 2)
                    
                    # Get depth at human center (handle NaN)
                    if (0 <= center_y < camera.height and 
                        0 <= center_x < camera.width):
                        distance = camera.depth_image[center_y, center_x]
                        
                        if not np.isnan(distance) and distance > 0:
                            if distance < min_distance:
                                min_distance = distance
                                closest_bbox = bbox
    
    # FEAR BEHAVIOR - graduated response based on closest human
    if human_detected and min_distance != float('inf'):
        # Determine if human is on left or right side
        bbox_center_x = (closest_bbox[0] + closest_bbox[2]) / 2
        frame_center_x = camera.width / 2
        human_on_left = bbox_center_x < frame_center_x
        
        if min_distance < 500:
            # Very close - retreat and rotate away
            robot.backward(0.6)
            if human_on_left:
                robot.spinRight(0.3)
            else:
                robot.spinLeft(0.3)
            cv2.putText(depth_colormap, f'DANGER {min_distance:.0f}mm - RETREAT!', 
                       (100, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)
            print(f'DANGER {min_distance:.0f}mm - Fast retreat & rotate', end='\r')
            
        elif min_distance < 900:
            # Medium distance - retreat
            robot.backward(0.4)
            cv2.putText(depth_colormap, f'CAUTION {min_distance:.0f}mm', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 165, 255), 2, cv2.LINE_AA)
            print(f'CAUTION {min_distance:.0f}mm - Medium retreat', end='\r')
            
        elif min_distance < 1300:
            # Far but still cautious - slow retreat
            robot.backward(0.2)
            cv2.putText(depth_colormap, f'ALERT {min_distance:.0f}mm', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2, cv2.LINE_AA)
            print(f'ALERT {min_distance:.0f}mm - Slow retreat', end='\r')
            
        else:
            # Safe distance - stop
            robot.stop()
            cv2.putText(depth_colormap, f'SAFE {min_distance:.0f}mm', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)
            print(f'SAFE {min_distance:.0f}mm - Human at safe distance', end='\r')
    else:
        # No human detected - stop
        robot.stop()
        cv2.putText(depth_colormap, 'SAFE - No human detected', 
                   (120, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)
        print('SAFE - No human threat detected', end='\r')
    
    # Display images
    scale = 0.1
    resized_color = cv2.resize(frame, None, fx=scale, fy=scale, 
                               interpolation=cv2.INTER_AREA)
    resized_depth = cv2.resize(depth_colormap, None, fx=scale, fy=scale, 
                               interpolation=cv2.INTER_AREA)
    display_color.value = bgr8_to_jpeg(resized_color)
    display_depth.value = bgr8_to_jpeg(resized_depth)

# Attach callback to camera
camera.observe(fear_callback, names=['color_value'])

**To stop the camera and robot:**

In [ ]:
camera.stop()

**Behavior Description:**

The robot exhibits fear behavior specifically toward humans:

1. **Distance Measurement**: Calculates distance to each detected human using depth image at bounding box center
2. **Graduated Fear Response**:
   - **< 500mm**: Fast backward retreat (0.6 speed) + rotate away from human
   - **< 900mm**: Medium backward retreat (0.4 speed)
   - **< 1300mm**: Slow backward retreat (0.2 speed)
   - **>= 1300mm**: Stop - safe distance
3. **Rotation Logic**: When very close (<500mm), robot rotates away from human's position (left or right)
4. **No Human**: Robot stops when no humans detected

The robot only fears humans, ignoring other objects in the environment.